Data Validation Script

In [2]:
import os

os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    RQD_FILE: list

In [4]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

class ConfigManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            STATUS_FILE=config.STATUS_FILE,
            RQD_FILE=config.RQD_FILE
        )

        return data_validation_config

In [7]:
import os
from TextSummarizer.logging import logger
import pathlib


class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

        # Create the status file if it doesn't exist
        status_file_path = pathlib.Path(self.config.STATUS_FILE)
        if not status_file_path.exists():
            status_file_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure directory exists
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write("Validation Status: Initialized\n")  # Initialize the file

    def validate_files_exist(self) -> bool:
        try:
            validation_status = None
            all_files = os.listdir(os.path.join("artifacts", "data_ingestion", "samsum_dataset"))

            for file in all_files:
                if file not in self.config.RQD_FILE:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}\n")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}\n")
            return validation_status
        except Exception as e:
            raise e

In [8]:
try:
    config = ConfigManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_files_exist()
except Exception as e:
    raise e

[2025-09-15 00:57:18,207: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2025-09-15 00:57:18,215: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-09-15 00:57:18,217: INFO: common: Created directory at: artifacts:]
[2025-09-15 00:57:18,224: INFO: common: Created directory at: data_validation:]
